# db-import

This is a dirty script to convert a telegram export into a json you can import into the database using `import_sumissions.py`.

It is ugly, but it works. :)

You need `spotipy` available to your jupyter kernel.

In [ ]:
# input your vars here here
spotipy_client_id = None
spotipy_client_secret = None
telegram_export_file = None
bot_name = None

In [ ]:
from datetime import datetime
from spotipy.client import Spotify
from spotipy.oauth2 import SpotifyClientCredentials

spotipy_provider = Spotify(
    client_credentials_manager=SpotifyClientCredentials(
        client_id=spotipy_client_id,
        client_secret=spotipy_client_secret,
    )
)

In [ ]:
import json
with open(telegram_export_filename) as f:
    data = json.load(f)

In [ ]:
botmsgs = [m for m in data["messages"] if m.get("from") == bot_name]
musicmsgs = [m for m in botmsgs if "sent by" in str(m.get("text", "")).lower()]

In [ ]:
len(musicmsgs)

In [ ]:
def extract_hashtags(elem):
    hashtags = []
    for i in elem.get("text"):
        if type(i) is dict:
            if "type" in i.keys():
                if i["type"] == "hashtag" and not "released_" in i["text"]:
                    hashtags.append(i["text"])
    return hashtags

In [ ]:
def get_data_from_spotify(like, url):
    spotid = url.split("/")[-1]
    
    if like == "/track":
        data = spotipy_provider.track(spotid)
        return (data["name"], data["external_urls"]["spotify"])
    
    if like == "/artist":
        data = spotipy_provider.track(spotid)
        return (data["album"]["artists"][0]["name"], data["album"]["artists"][0]["external_urls"]["spotify"])
    
    if like == "/album":
        data = spotipy_provider.track(spotid)
        return (data["album"]["name"], data["album"]["external_urls"]["spotify"])

    if like == "year":
        data = spotipy_provider.album(spotid)
        release_date = data["release_date"]
        return datetime.fromisoformat(f"{release_date.split('-')[0]}-01-01")
    

    return ("NOTFOUND", url)

In [ ]:
def get_link_tuple(elem, like):
    for i in elem.get("text"):
        if type(i) is dict and i.get("text") == "Spotify":
            spotify_url = i["href"]
        if type(i) is dict and "type" in i.keys() and i["type"] == "text_link" and like in i["href"] and i["text"] != "Spotify":
            return (i["text"], i["href"])
    return get_data_from_spotify(like, spotify_url)

In [ ]:
def get_year(elem, url):
    for i in elem.get("text"):
        if type(i) is dict and "type" in i.keys() and i["type"] == "hashtag" and "#released" in i["text"]:
            return datetime(int(i["text"].split("_")[-1]), 1, 1)
    return get_data_from_spotify('year', url)

In [ ]:
def get_track_youtube_url(elem, track_name):
    for i in elem.get("text"):
        if type(i) is dict and "type" in i.keys() and i["type"] == "text_link" and (track_name in i["text"] or i["text"] == "YouTube"):
            return i["href"]

In [ ]:
def get_dj(elem):
    for i in elem.get("text"):
        if type(i) is dict and "type" in i.keys() and i["type"] == "mention":
            return i["text"]

In [ ]:
submissions = []

for m in musicmsgs:
    text_str = str(m.get("text", "")).lower()
    artist_name, artist_url = get_link_tuple(m, "/artist")
    dj = get_dj(m)
    
    submission = {
        "dj": dj[1:] if dj[0] == "@" else dj(m),
        "submission_date": "{datetimeserializer}:" + datetime.fromisoformat(m.get("date", "2020-01-01")).isoformat(),
        "submission_type": "{submissiontypeserializer}:artist",
        "artist_name": artist_name,
        "artist_url": artist_url,
        "artist_genre_tags": extract_hashtags(m),
        "album_name": None,
        "album_url": None,
        "album_release_date": None,
        "track_name": None,
        "track_url": None,
        "track_url_youtube": None,
    }
    
    if "album sent" in text_str or "track sent" in text_str:
        album_name, album_url = get_link_tuple(m, "/album")
        submission["submission_type"] = "{submissiontypeserializer}:album"
        submission["album_name"] = album_name
        submission["album_url"] = album_url
        submission["album_release_date"] = get_year(m, album_url)
        
    if "track sent" in text_str:
        track_name, track_url = get_link_tuple(m, "/track")
        track_youtube_url = get_track_youtube_url(m, track_name)
        submission["submission_type"] = "{submissiontypeserializer}:track"
        submission["track_name"] = track_name
        submission["track_url"] = track_url
        submission["track_url_youtube"] = track_youtube_url
    
    submissions.append(submission)

In [ ]:
submissions